<div>
    <table>
        <tr>
            <td>
                <center>
                    <h1>Brightway Introduction</h1>
                     <a href="https://www.psi.ch/en/ta/people/romain-sacchi">Romain Sacchi</a> (PSI)
                    <br><br>
                    Duration: 1 hour 30 minutes.
                </center>
            </td>
        </tr>
    </div>

# Brightway I/O: bw2io

<div class="alert alert-info">
Note: we will be using <a href="https://docs.brightway.dev/en/legacy/">Brightway 2</a>, not <a href="https://docs.brightway.dev/en/latest/content/installation/index.html">Brightway 2.5</a>. From the user end side, very little differs between the two. The code executed throughout this notebook works with both versions.
</div>


## Learning objectives  

Learn how to:

    - input LCI data to Brightway using Excel/CVS importers
    - fix linking issue using migration files
    - do a contribution analysis
    - export your foreground inventories

## Standard inputs and setup

In [1]:
import os
from pathlib import Path
import pandas as pd
import bw2io, bw2data, bw2calc

Let's list our projects:

In [2]:
list(bw2data.projects)

[Project: ,
 Project: bw25_ei310,
 Project: default,
 Project: ecoinvent-migration,
 Project: ecoinvent-3.9.1-cutoff_bw25,
 Project: ecoinvent-3.9.11-cutoff,
 Project: aligned_bw25,
 Project: ecoinvent-3.10-cutoff,
 Project: defult,
 Project: winter-school,
 Project: test,
 Project: training-day,
 Project: 66,
 Project: 74,
 Project: training-day-2]

Setting the project

In [3]:
bw2data.projects.set_current("training-day-2")

## Context

Performing an LCA generally requires:
  - Background LCI data (e.g., an LCI database such as [ecoinvent](https://ecoinvent.org/))  
  - Foreground LCI data (e.g., a bunch of datasets the LCA practitioner has spent time modeling)
  - Sets of characterization factors.
 
This section will deal with the way Foreground LCI data is input to Brightway.

Useful documentation about what a database in Brightway is can be found [here](https://github.com/brightway-lca/brightway2/blob/master/notebooks/Databases.ipynb)
 and [here](https://docs.brightway.dev/en/latest/content/gettingstarted/databases.html).

# Importing from CSV or Excel

Using `bw2io.ExcelImporter`, we import datasets from an Excel file.

In [8]:
imp = bw2io.ExcelImporter("files/lci-carbon-fiber.xlsx")

Extracted 1 worksheets in 0.05 seconds


In [5]:
from constructive_geometries import geomatcher

In [6]:
geo = geomatcher.Geomatcher()

In [7]:
geo.contained("RER")

[('ecoinvent', 'RER'),
 ('ecoinvent', 'Europe without Austria'),
 ('ecoinvent', 'Europe without Switzerland'),
 ('ecoinvent', 'Europe without Switzerland and Austria'),
 ('ecoinvent', 'RER w/o CH+DE'),
 ('ecoinvent', 'Europe, without Russia and Türkiye'),
 ('ecoinvent', 'RER w/o RU'),
 ('ecoinvent', 'Europe without Switzerland and France'),
 ('ecoinvent', 'RER w/o DE+NL+RU'),
 ('ecoinvent', 'RER w/o DE+NL+NO'),
 ('ecoinvent', 'ENTSO-E'),
 ('ecoinvent', 'Europe without NORDEL (NCPA)'),
 ('ecoinvent', 'RER w/o DE+NL+NO+RU'),
 ('ecoinvent', 'RER w/o AT+BE+CH+DE+FR+IT'),
 ('ecoinvent', 'UCTE'),
 ('ecoinvent', 'IAI Area, EU27 & EFTA'),
 ('ecoinvent', 'UCTE without Germany'),
 ('ecoinvent', 'UCTE without France'),
 ('ecoinvent', 'UCTE without Germany and France'),
 ('ecoinvent', 'UN-SEUROPE'),
 ('ecoinvent', 'UN-NEUROPE'),
 'GB',
 ('ecoinvent', 'WEU'),
 ('ecoinvent', 'NORDEL'),
 'SI',
 ('ecoinvent', 'BALTSO'),
 'IT',
 ('ecoinvent', 'Russia (Europe)'),
 'LV',
 'NO',
 'FR',
 'GR',
 'MK',
 ('ec

We want to apply a number of data cleaning functions (format numbers, set correct location, etc.),

In [9]:
imp.apply_strategies()

Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 2.48 seconds


Then, we want to use the `match_database()` method to link exchanges to suppliers.

First, we want to link exchange to suppliers that may also be contained in the data being imported.

In [10]:
# we match based on the name, reference product and location
imp.match_database(fields=('name', 'reference product', 'unit', 'location')) 

Applying strategy: link_iterable_by_fields


<div class="alert alert-info">
Note: Why is it important to link both based on <b>name</b> and <b>reference product</b>?
</div>

Is that enough? Do we still have unlinked exchanges? Let's check.

In [11]:
imp.statistics()

10 datasets
109 exchanges
82 unlinked exchanges
  Type biosphere: 1 unique unlinked exchanges
  Type technosphere: 19 unique unlinked exchanges


(10, 109, 82)

Let's check what those unlinked exchanges are:

In [12]:
for u in list(imp.unlinked):
    print(u["name"], u.get("location"), u.get("categories"))

market for heat, from steam, in chemical industry RER None
market for acrylonitrile GLO None
market group for electricity, low voltage RER None
market for methyl acrylate GLO None
market for acrylic acid RER None
market for water, deionised Europe without Switzerland None
market for compressed air, 1000 kPa gauge RER None
market for dimethyl sulfoxide GLO None
market for ethylene glycol GLO None
air separation, cryogenic RER None
market for steam, in chemical industry RER None
treatment of wastewater, average, wastewater treatment CH None
market for potassium permanganate GLO None
market for silicone product RER None
Argon-40 None ('air',)
market for natural gas, medium pressure, vehicle grade GLO None
market for NOx retained, by selective catalytic reduction GLO None
market for tap water Europe without Switzerland None
market for ammonium bicarbonate RER None
market for epoxy resin, liquid RER None


OK, some unlinked exchanges are clearly from ecoinvent. Let's try to link those.

In [13]:
imp.match_database("ecoinvent-3.10-cutoff", fields=('name', 'reference product', 'unit', 'location'))
imp.statistics()

Applying strategy: link_iterable_by_fields
10 datasets
109 exchanges
14 unlinked exchanges
  Type biosphere: 1 unique unlinked exchanges
  Type technosphere: 2 unique unlinked exchanges


(10, 109, 14)

Depiste trying to link with ecoinvent, we still have two unmatched technosphere flows:

In [14]:
[u for u in imp.unlinked if u["type"] == "technosphere"]

[{'name': 'market for ethylene glycol',
  'amount': 2.4225526641883498e-06,
  'database': 'ecoinvent',
  'location': 'GLO',
  'unit': 'kilogram',
  'type': 'technosphere',
  'reference product': 'ethylene glycol'},
 {'name': 'air separation, cryogenic',
  'amount': 0.005396530359355638,
  'database': 'ecoinvent',
  'location': 'RER',
  'unit': 'kilogram',
  'type': 'technosphere',
  'reference product': 'nitrogen, liquid'}]

Also, we have an unlinked biosphere exchange left, let's try to match that one.

In [15]:
imp.match_database("biosphere", fields=('name', 'unit', 'categories'))
imp.statistics()

Applying strategy: link_iterable_by_fields
10 datasets
109 exchanges
14 unlinked exchanges
  Type biosphere: 1 unique unlinked exchanges
  Type technosphere: 2 unique unlinked exchanges


(10, 109, 14)

In [16]:
list(imp.unlinked)

[{'name': 'market for ethylene glycol',
  'amount': 2.4225526641883498e-06,
  'database': 'ecoinvent',
  'location': 'GLO',
  'unit': 'kilogram',
  'type': 'technosphere',
  'reference product': 'ethylene glycol'},
 {'name': 'air separation, cryogenic',
  'amount': 0.005396530359355638,
  'database': 'ecoinvent',
  'location': 'RER',
  'unit': 'kilogram',
  'type': 'technosphere',
  'reference product': 'nitrogen, liquid'},
 {'name': 'Argon-40',
  'amount': 0.012598425196850395,
  'unit': 'kilogram',
  'categories': ('air',),
  'type': 'biosphere',
  'uncertainty type': 2,
  'loc': -4.3741834572128555,
  'u1': 1.5,
  'u2': 1.2,
  'u3': 1.5,
  'u4': 1.1,
  'u5': 2,
  'u6': 1.2,
  'ub': 1.5,
  'scale': 0.5121584730617011}]

In [17]:
[u for u in imp.unlinked if u["type"] == "biosphere"]

[{'name': 'Argon-40',
  'amount': 0.012598425196850395,
  'unit': 'kilogram',
  'categories': ('air',),
  'type': 'biosphere',
  'uncertainty type': 2,
  'loc': -4.3741834572128555,
  'u1': 1.5,
  'u2': 1.2,
  'u3': 1.5,
  'u4': 1.1,
  'u5': 2,
  'u6': 1.2,
  'ub': 1.5,
  'scale': 0.5121584730617011}]

Nope. Why not? Maybe because `Argon-40` does not not exist as such in `biosphere?`

In [18]:
[f for f in bw2data.Database("biosphere") if "argon" in f["name"].lower()]

['Argon' (kilogram, None, ('natural resource', 'in air')),
 'Argon-41' (kilo Becquerel, None, ('air', 'non-urban air or from high stacks')),
 'Argon-41' (kilo Becquerel, None, ('air', 'urban air close to ground')),
 'Argon' (kilogram, None, ('air',)),
 'Argon-41' (kilo Becquerel, None, ('air',)),
 'Argon-41' (kilo Becquerel, None, ('air', 'low population density, long-term'))]

It is indeed now simply called `Argon` in ecoinvent 3.10.
We can:
1. manually fix this (i.e., modify the exchange name in the Excel file),
2. go over `imp.data`(list), iterate through the exchanges and find `Argon-40` and replace it with `Argon`
3. create a `migration` file for translating ecoinvent 3.9 flows to 3.10

### Migration file

We create a mapping dictionary, and use it to create a `Migration` object.

In [19]:
migration = {
    "fields": ["name", "reference product", "location", "categories"],
    "data": [
        (
            ("market for ethylene glycol", "ethylene glycol", "GLO", ""),
            {"location": "RER",},
        ),
        (
            ("air separation, cryogenic", "nitrogen, liquid", "GLO", ""),
            {
                "name": "industrial gases production, cryogenic air separation",
                "location": "RER",
            },
        ),
        (
            ("air separation, cryogenic", "nitrogen, liquid", "RER", ""),
            {
                "name": "industrial gases production, cryogenic air separation",
            },
        ),
        (
            ("Argon-40", "", "", ("air",)),
            {
                "name": "Argon",
            },
        )
    ],
}

In [20]:
bw2io.Migration(name="ei3.9-3.10").write(data=migration, description="ei 3.9 to 3.10")

In [21]:
"ei3.9-3.10" in bw2io.migrations

True

In [22]:
bw2io.Migration("ei3.9-3.10")

Brightway2 Migration: ei3.9-3.10

We apply the migration on our imported data.

In [23]:
imp.data = bw2io.strategies.migrate_exchanges(
    db=imp.data,
    migration="ei3.9-3.10"
)

In [24]:
imp.match_database("ecoinvent-3.10-cutoff", fields=('name', 'reference product', 'unit', 'location'))
imp.match_database("biosphere", fields=('name', 'unit', 'categories'))
imp.statistics()

Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
10 datasets
109 exchanges
0 unlinked exchanges
  


(10, 109, 0)

We have zero unlinked exchanges, we're ready to write the database.

In [25]:
if len(list(imp.unlinked)) == 0:
    imp.write_database()

Writing activities to SQLite3 database:
0% [##########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 04/03/2025 13:54:12
  Finished: 04/03/2025 13:54:12
  Total time elapsed: 00:00:00
  CPU %: 110.40
  Memory %: 2.13
Created database: carbon fiber


In [26]:
bw2data.databases

Databases dictionary with 3 object(s):
	biosphere
	carbon fiber
	ecoinvent-3.10-cutoff

# Contribution analyses

## Process contribution

We have already seen how to obtain a contribution analysis in terms of contributing processes:

In [27]:
db = bw2data.Database("carbon fiber")

In [28]:
# let's list the datasets in our new database "carbon fiber"
[a["name"] for a in db]

['carbon fiber production, fiber coagulation, stretching, washing, sizing and drying',
 'carbon fiber production, exhaust gas treatment 2',
 'carbon fiber production, fiber stabilization, carbonization, electrolysis and washing',
 'carbon fiber production, fiber drying and sizing',
 'carbon fiber production, exhaust gas treatment 1',
 'carbon fiber production, fiber relaxation',
 'polyacrylonitrile production (PAN) by polymerisation',
 'carbon fiber production, weaved, at factory',
 'carbon fiber production, fiber winding and unwinding',
 'Dimethyl sulfoxide production (DMSO)']

In [29]:
activity = db.search('carbon fiber production, weaved, at factory')[0]
activity

'carbon fiber production, weaved, at factory' (kilogram, RER, None)

In [30]:
method = ('IPCC 2021', 'climate change', 'global warming potential (GWP100)')

In [31]:
lca = bw2calc.LCA({activity:1}, method)
lca.lci()
lca.lcia()
lca.score

/opt/homebrew/Caskroom/miniforge/base/envs/bw2/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 1.21e+13)
  warnings.warn(msg, UmfpackWarning)


75.11270153072913

In [32]:
rev_prod, rev_act, rev_bio = lca.reverse_dict()

In [33]:
results_by_activity = (lca.characterized_inventory.sum(axis=0)).A1

In [34]:
# Create a list of names in columns
list_of_names_in_columns = [
    bw2data.get_activity(rev_prod[col])['name'] 
    for col in range((lca.characterized_inventory.sum(axis=0)).shape[1])
]

In [35]:
pd.Series(index=list_of_names_in_columns, data=results_by_activity).sort_values(ascending=False).head(10)

heat production, natural gas, at industrial furnace >100kW    8.068948
carbon fiber production, exhaust gas treatment 2              7.448622
natural gas venting from petroleum/natural gas production     4.314160
heat production, at hard coal industrial furnace 1-10MW       2.413090
ammonia production, partial oxidation, liquid                 2.203817
ammonia production, steam reforming, liquid                   2.094031
ammonia production, partial oxidation, liquid                 1.906205
heat production, at hard coal industrial furnace 1-10MW       1.796146
electricity production, lignite                               1.766702
sweet gas, burned in gas turbine                              1.513273
dtype: float64

With BW2.5 But there is a simpler and more "official" way to obtain this.

In [36]:
import bw2analyzer as ba

In [37]:
pd.DataFrame(
    [(x, y, z["unit"], z["name"]) for x, y, z in ba.ContributionAnalysis().annotated_top_processes(lca=lca)],
    columns=["score", "quantity", "unit", "name"]
)

,score,quantity,unit,name
0,8.068948,136.671830,megajoule,"heat production, natural gas, at industrial fu..."
1,7.448622,10.609957,kilogram,"carbon fiber production, exhaust gas treatment 2"
2,4.314160,0.247272,cubic meter,natural gas venting from petroleum/natural gas...
3,2.413090,20.967055,megajoule,"heat production, at hard coal industrial furna..."
4,2.203817,0.787078,kilogram,"ammonia production, partial oxidation, liquid"
5,2.094031,1.461715,kilogram,"ammonia production, steam reforming, liquid"
6,1.906205,0.680788,kilogram,"ammonia production, partial oxidation, liquid"
7,1.796146,15.606498,megajoule,"heat production, at hard coal industrial furna..."
8,1.766702,1.490596,kilowatt hour,"electricity production, lignite"
9,1.513273,22.596866,megajoule,"sweet gas, burned in gas turbine"


## Importing custom impact assessment methods

Let's import an Excel file containing names of greenhouse gases and factors (according to the IPCC 2021 GWP method), and add a CF for Hydrogen (11 kg $CO_2$-eq./kg $H_2$)

In [38]:
from bw2io import ExcelLCIAImporter
new_method = ExcelLCIAImporter(
    filepath="files/lcia_gwp2021_100a_w_bio.xlsx",
    name=("IPCC 2021", "GWP 100 with bio C and H2"),
    unit="kg CO2e",
    description="modified IPCC GWP100 method"
)

In [39]:
new_method.apply_strategies()

Applying strategy: csv_restore_tuples
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applied 8 strategies in 0.08 seconds


In [40]:
new_method.name = ("IPCC 2021", "GWP 100 with bio C and H2")

In [41]:
new_method.write_methods(
    overwrite=True,
    verbose=True,
)

Wrote 1 LCIA methods with 34 characterization factors


In [42]:
("IPCC 2021", "GWP 100 with bio C and H2") in bw2data.methods

True

In [43]:
act=bw2data.Database("ecoinvent-3.10-cutoff").random()
for method in [
    ('IPCC 2021', 'climate change', 'global warming potential (GWP100)'),
    ("IPCC 2021", "GWP 100 with bio C and H2")
]:
    test = bw2calc.LCA({act: 1}, method)
    test.lci()
    test.lcia()
    print(test.score)

/opt/homebrew/Caskroom/miniforge/base/envs/bw2/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 1.21e+13)
  warnings.warn(msg, UmfpackWarning)


138461785.0644299
130294712.36247818


We can also build a method manually.

In [45]:
data = [
    [f.key, 1]
    for f in bw2data.Database("biosphere")
    if "carbon dioxide" in f["name"].lower()
]
data

[[('biosphere', 'eba59fd6-f37e-41dc-9ca3-c7ea22d602c7'), 1],
 [('biosphere', 'e259263c-d1f1-449f-bb9b-73c6d0a32a00'), 1],
 [('biosphere', 'e8787b5e-d927-446d-81a9-f56977bbfeb4'), 1],
 [('biosphere', '60d424f7-d5a9-4549-9540-da06684bc3bb'), 1],
 [('biosphere', '375bc95e-6596-4aa1-9716-80ff51b9da77'), 1],
 [('biosphere', 'd6235194-e4e6-4548-bfa3-ac095131aef4'), 1],
 [('biosphere', 'e1c597cc-14cb-4ebb-af07-7a93a5b77d34'), 1],
 [('biosphere', '259cf8d6-6ea8-4ccf-84b7-23c930a5b2b3'), 1],
 [('biosphere', '8ae4d8bb-3e4b-4825-8325-94696d7a64fd'), 1],
 [('biosphere', '28e1e2d6-97ad-4dfd-932a-9edad36dcab9'), 1],
 [('biosphere', '73ed05cc-9727-4abf-9516-4b5c0fe54a16'), 1],
 [('biosphere', 'cc6a1abb-b123-4ca6-8f16-38209df609be'), 1],
 [('biosphere', '78eb1859-abd9-44c6-9ce3-f3b5b33d619c'), 1],
 [('biosphere', '349b29d1-3e58-4c66-98b9-9d1a076efd2e'), 1],
 [('biosphere', 'f9749677-9c9f-4678-ab55-c607dfdc2cb9'), 1],
 [('biosphere', '419de9f0-ee00-4e95-9556-c8f06b17beec'), 1],
 [('biosphere', '16eeda8

In [46]:
my_method = bw2data.Method(("IPCC 2021", "Simple CO2 IA method"))
my_method.validate(data)
my_method.register()
my_method.metadata = {"unit": "kg CO2eq.", "abbreviation": "GWP"}
my_method.write(data)

In [47]:
method = ("IPCC 2021", "Simple CO2 IA method")
method in bw2data.methods

True

In [48]:
test = bw2calc.LCA({act: 1}, method)
test.lci()
test.lcia()
print(test.score)

149510470.77643278


## Tree map

In [49]:
from polyviz import treemap

In [50]:
method = ('IPCC 2021', 'climate change', 'global warming potential (GWP100)')
treemap(
    activity=activity,
    method=method
)

Calculating LCIA score...


/opt/homebrew/Caskroom/miniforge/base/envs/bw2/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 1.21e+13)
  warnings.warn(msg, UmfpackWarning)


'/Users/romain/GitHub/training-day-dtu-2025/tutorials/brightway/carbon fiber production weaved at factory kilogram RER IPCC 2021climate changeglobal warming potential GWP100 treemap.html'

## Supply chain traversal

In [51]:
from polyviz import sankey

Github repo: [link](https://github.com/romainsacchi/polyviz). No proper documentation yet, but a notebook with [examples](https://github.com/romainsacchi/polyviz/blob/main/examples/examples.ipynb).

In [52]:
_, df = sankey(
    activity=activity,
    level=4,
    cutoff=0.01,
    method=method,
    labels_swap={
        "carbon fiber": "cf.",
        "production": "prod."
    }
)

Calculating supply chain score...


/opt/homebrew/Caskroom/miniforge/base/envs/bw2/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 1.21e+13)
  warnings.warn(msg, UmfpackWarning)


Sankey diagram generated.


## Violin plot

In [53]:
from polyviz import violin
import warnings
warnings.filterwarnings("ignore")

In [54]:
method = ('IPCC 2021', 'climate change', 'global warming potential (GWP100)')
violin(
    activities=[
        a for a in bw2data.Database("ecoinvent-3.10-cutoff") 
        if a["unit"] == "ton kilometer"
    ][:3],
    method=method,
    iterations=10
)

/opt/homebrew/Caskroom/miniforge/base/envs/bw2/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 1.27e+14)
  warnings.warn(msg, UmfpackWarning)
/opt/homebrew/Caskroom/miniforge/base/envs/bw2/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 6.39e+14)
  warnings.warn(msg, UmfpackWarning)
/opt/homebrew/Caskroom/miniforge/base/envs/bw2/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 1.12e+14)
  warnings.warn(msg, UmfpackWarning)
/opt/homebrew/Caskroom/miniforge/base/envs/bw2/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 1.11e+14)
  warnings.warn(msg, UmfpackWarning)
/opt/homebrew/Caskroom/miniforge/base/envs/bw2/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: Umfp

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
pivottablejs

# Exporting databases

## To Excel

We can export the entire database inventory to an Excel file.

In [ ]:
bw2io.export.write_lci_excel?

In [55]:
bw2io.export.write_lci_excel(
    database_name="carbon fiber",
)

'/Users/romain/Library/Application Support/Brightway3/training-day-2.cf3037efa523ef612ceb85312be46ddb/output/lci-carbon-fiber.xlsx'

## As a bw2package file

We can also export the database as a Brightway package file.

In [56]:
bw2io.package.BW2Package().export_obj(
    obj=db,
)

'/Users/romain/Library/Application Support/Brightway3/training-day-2.cf3037efa523ef612ceb85312be46ddb/export/carbon-fiber54c4141962fcd79bed713eb04066d118.f3770bd032cc888161f5e9553d478654.bw2package'

In [ ]:
bw2io.package.BW2Package().import_file("filepath to bw2package")

<div class="alert alert-info">
Note: It may not be ideal for sharing, because for the import to be successful, the other user will need the databases the exported database depends on (ecoinvent, biosphere) to be named exactly the same. It is still possible, though, but the user you share the package with, will have to correct this upon import.
</div>

## As a project

We can export the entire project. This is the safest option, as all the database `carbon fiber` depends on are also exported. The drawback is that the file is bigger, and there may be licensing issues. But it is at least a good backup solution.

In [ ]:
bw2data.projects.current

In [ ]:
bw2io.backup.backup_project_directory?

In [ ]:
bw2data.projects.set_current("training-day")
bw2data.databases

In [57]:
bw2io.backup.backup_project_directory(
    project='training-day-2',
)

Creating project backup archive - this could take a few minutes...


'training-day-2'

And we load it back up... Note that I give it another name to not overwrite it.
Also, overwrite is `False` by default, so it would need ot be set to `True` first.

In [58]:
bw2io.backup.restore_project_directory(
    fp="/Users/romain/brightway2-project-training-day-2-backup.03-April-2025-02-09PM.tar.gz",
    project_name="training-day-3",
    overwrite_existing=False
)

Restoring project backup archive - this could take a few minutes...


'training-day-3'

Let's check

In [59]:
"training-day-3" in bw2data.projects

True

In [60]:
bw2data.projects.delete_project("training-day-3", delete_dir=True)

'training-day-2'